# Capstone Project - The Battle of Neighborhoods

## PROJECT CONTENT

1. Introduction Section : ⁃ Discussion of the business problem.
2. Data Section: ⁃ Description of the data that will be used to solve the problem and the sources.
3. Methodology section ⁃ Discussion and description of exploratory data analysis carried out, any inferential statistical testing performed, and if any machine learnings were used establishing the strategy and purposes.
4. Results section ⁃ Discussion of the results.
5. Discussion section ⁃ Elaboration and discussion on any observations noted and any recommendations suggested based on the results.
6. Conclusion section ⁃ Report Conclusion.

## 1.Introduction Section:

Discussion of the business problem and the interested audience in this project.

Description of the problem and background.

Scenario:

I am choosing which city should I go to study in, Syndey or Melbourne. 

The deciding factors are as follows:

##### Which city has more venues around university?





## 2.Data Section

Description of the data that will be used to solve the problem and the sources.

The following data is required to answer the issue of the problem:

#####  The number of venues in Downtown Syndey and Melbourne.

How the data will be used to solve the problem?

The data will be used as follows:

##### Use Foursquare and geopy to account the number of venues around the USYD and the university of Melbourne.


### Code

In [1]:
!pip install folium
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

import folium # map rendering library

print('Libraries imported.')

Libraries imported.


#### The geograpical coordinate of the university of Melbourne

In [2]:
address = "Parkville VIC 3010, Australia "

geolocator = Nominatim(user_agent="Melbourne_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of the university of Melbourne are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of the university of Melbourne are -37.79979885, 144.95861200950955.


In [3]:
map_Melbourne = folium.Map(location=[latitude, longitude], zoom_start=11)
    
map_Melbourne

In [4]:
neighborhood_latitude=-33.8896116
neighborhood_longitude=151.1800986

#### Use Foursquare

In [23]:
CLIENT_ID = '' # Due to the privacy, deleted
CLIENT_SECRET = '' 
VERSION = '20210724' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 
CLIENT_SECRET:


#### Melbourne results

In [24]:
LIMIT = 100# limit of number of venues returned by Foursquare API
radius = 500# define radiu
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=&client_secret=&v=20210724&ll=-33.8896116,151.1800986&radius=500&limit=100'

In [7]:
import requests
Mel_results = requests.get(url).json()
Mel_results

{'meta': {'code': 200, 'requestId': '60fbe39bf3d1e20203ab527c'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Parkville',
  'headerFullLocation': 'Parkville, Melbourne',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 43,
  'suggestedBounds': {'ne': {'lat': -37.7952988455, 'lng': 144.96429645139102},
   'sw': {'lat': -37.804298854500004, 'lng': 144.95292756762808}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '4d9d3d8e5c33a35d92a7fda0',
       'name': 'Carte Crêpes',
       'location': {'address': 'University of Melbourne',
        'lat': -37.79905216385439,
        'lng': 144.96035697797498,
        'labeledLatLngs': [{'label': 'display',
          'lat': -3

In [8]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [9]:
from pandas.io.json import json_normalize

Mel_venues = Mel_results['response']['groups'][0]['items']
    
Melnearby_venues = json_normalize(Mel_venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
Melnearby_venues =Melnearby_venues.loc[:, filtered_columns]

# filter the category for each row
Melnearby_venues['venue.categories'] = Melnearby_venues.apply(get_category_type, axis=1)

# clean columns
Melnearby_venues.columns = [col.split(".")[-1] for col in Melnearby_venues.columns]

Melnearby_venues.head(10)

<ipython-input-9-dafb4e1d6fbc>:5: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  Melnearby_venues = json_normalize(Mel_venues) # flatten JSON


,name,categories,lat,lng
0,Carte Crêpes,Café,-37.799052,144.960357
1,Seven Seeds,Café,-37.802870,144.959120
2,Lanna Thai,Thai Restaurant,-37.799097,144.954388
3,House of Cards,Coffee Shop,-37.799243,144.961597
4,University House,Lounge,-37.796282,144.960054
5,Egg Sake Bistro,Japanese Restaurant,-37.797019,144.961163
6,Arabesque,Persian Restaurant,-37.803363,144.958384
7,Bobbie Peels,Bar,-37.803849,144.958320
8,Arts House Meat Market,Indie Theater,-37.801289,144.954410
9,Dr Dax Kitchen,Coffee Shop,-37.798186,144.957974


In [10]:

map_mel = folium.Map(location=[latitude, longitude], zoom_start=20)

# add markers to map
for lat, lng, label in zip(Melnearby_venues['lat'], Melnearby_venues['lng'], Melnearby_venues['name']):
    label = folium.Popup(label, parse_html=True)
    folium.RegularPolygonMarker(
        [lat, lng],
        number_of_sides=4,
        radius=10,
        popup=label,
        color='blue',
        fill_color='#0f0f0f',
        fill_opacity=0.7,
    ).add_to(map_mel)  
    
map_mel

In [11]:
print('{} venues were returned by Foursquare.'.format(Melnearby_venues.shape[0]))

43 venues were returned by Foursquare.


#### The geograpical coordinate of the university of syndey

In [12]:
address = "Camperdown NSW 2006, Australia "

geolocator = Nominatim(user_agent="Syndey_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of the university of syndey are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of the university of syndey are -33.8896116, 151.1800986.


In [13]:
map_Syndey = folium.Map(location=[latitude, longitude], zoom_start=11)
    
map_Syndey

In [21]:
CLIENT_ID = '' # Due to the privacy, deleted
CLIENT_SECRET = '' 
VERSION = '20210724' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 
CLIENT_SECRET:


#### Syndey results

In [22]:
LIMIT = 100# limit of number of venues returned by Foursquare API
radius = 500# define radiu
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    latitude, 
    longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=&client_secret=&v=20210724&ll=-33.8896116,151.1800986&radius=500&limit=100'

In [16]:
Syndey_results = requests.get(url).json()
Syndey_results

{'meta': {'code': 200, 'requestId': '60fbe39d1f13550515d60c57'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'Camperdown',
  'headerFullLocation': 'Camperdown, Sydney',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 25,
  'suggestedBounds': {'ne': {'lat': -33.8851115955, 'lng': 151.1855094300044},
   'sw': {'lat': -33.894111604500004, 'lng': 151.17468776999561}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '51140e47e4b02581fe436942',
       'name': 'IRO Café',
       'location': {'address': '2/35 Missenden Rd',
        'lat': -33.8878,
        'lng': 151.18017,
        'labeledLatLngs': [{'label': 'display',
          'lat': -33.8878,
          'lng': 151.18

In [17]:
def get_category_type(row):
    try:
        Syndey_categories_list = row['categories']
    except:
        Syndey_categories_list = row['venue.categories']
        
    if len(Syndey_categories_list) == 0:
        return None
    else:
        return Syndey_categories_list[0]['name']

In [18]:
Syndey_venues = Syndey_results['response']['groups'][0]['items']
    
Syndeynearby_venues = json_normalize(Syndey_venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
Syndeynearby_venues =Syndeynearby_venues.loc[:, filtered_columns]

# filter the category for each row
Syndeynearby_venues['venue.categories'] = Syndeynearby_venues.apply(get_category_type, axis=1)

# clean columns
Syndeynearby_venues.columns = [col.split(".")[-1] for col in Syndeynearby_venues.columns]

Syndeynearby_venues.head(10)

<ipython-input-18-e821a01b8941>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  Syndeynearby_venues = json_normalize(Syndey_venues) # flatten JSON


,name,categories,lat,lng
0,IRO Café,Café,-33.887800,151.180170
1,Gather On The Green,Café,-33.890266,151.176188
2,Acre Farm and Eatery,Restaurant,-33.888066,151.176885
3,Campos Coffee,Coffee Shop,-33.893007,151.183213
4,Konya Kebab,Burger Joint,-33.886960,151.177260
5,Alfred Hotel,Pub,-33.887966,151.180656
6,Camperdown Cellars,Liquor Store,-33.886644,151.177121
7,Papaya Thai,Thai Restaurant,-33.887466,151.180159
8,Deus Ex Machina,Motorcycle Shop,-33.886167,151.179266
9,Boss Lady Food & Co.,Café,-33.885624,151.178480


In [19]:
map_Syndey = folium.Map(location=[latitude, longitude], zoom_start=20)

# add markers to map
for lat, lng, label in zip(Syndeynearby_venues['lat'], Syndeynearby_venues['lng'], Syndeynearby_venues['name']):
    label = folium.Popup(label, parse_html=True)
    folium.RegularPolygonMarker(
        [lat, lng],
        number_of_sides=4,
        radius=10,
        popup=label,
        color='yellow',
        fill_color='#0f0f0f',
        fill_opacity=0.7,
    ).add_to(map_Syndey)  
    
map_Syndey

In [20]:
print('{} venues were returned by Foursquare.'.format(Syndeynearby_venues.shape[0]))

25 venues were returned by Foursquare.


### Results:

I will choose Melbourne.There are two reasons.
1. The number of venues
Melbourne has more venues than Syndey.
43 venues were returned by Foursquare in Melbourne.And 25 venues were returned by Foursquare in Syndey.
2. The distance between two venues
As we can see from the map, Melbourne's venues are more concentrated than Sydney's.